In [1]:
import pandas as pd # pandas dataframes
from sklearn.ensemble import RandomForestClassifier # model that takes in data to train and classify the imports
from sklearn.metrics import f1_score, make_scorer, classification_report 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder # one hot encoding of team names
import numpy as np
from collections import defaultdict
import time


In [2]:
# # import csv files from git repository
# teamPath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/teamData.csv'
# playerPath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/playerData.csv'
# outcomePath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/outcomeData.csv'

In [3]:
# Justin file paths
teamPath = '/Users/justinholmes/Desktop/Sports_Model/Data/teamData.csv'
playerPath = '/Users/justinholmes/Desktop/Sports_Model/Data/playerData.csv'
outcomePath = '/Users/justinholmes/Desktop/Sports_Model/Data/outcomeData.csv'
featurePath = '/Users/justinholmes/Desktop/Sports_Model/Data/featureData.csv'

In [4]:
# convert csv files into pandas dataframes
teamData = pd.read_csv(teamPath)
playerData = pd.read_csv(playerPath, engine='python')
outcomeData = pd.read_csv(outcomePath)

In [5]:
teamData

,Team,Year,G_PG,MP_PG,FG_PG,FGA_PG,FG%_PG,3P_PG,3PA_PG,3P%_PG,...,TOV%,ORB%,FT/FGA,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Arena,Attend.,Attend./G
0,Phoenix Suns*,2005,82,241.2,40.9,85.6,0.477,9.7,24.7,0.393,...,12.4,27.5,0.222,0.478,12.0,68.3,0.176,America West Arena,"726,066","17,709"
1,Sacramento Kings*,2005,82,242.1,39.1,85.1,0.459,6.4,17.0,0.374,...,12.0,28.8,0.226,0.493,13.3,69.0,0.214,ARCO Arena (II),"709,997","17,317"
2,Dallas Mavericks*,2005,82,240.6,37.3,81.6,0.457,5.6,15.5,0.364,...,12.5,28.6,0.273,0.472,14.3,70.2,0.239,American Airlines Center,"822,533","20,062"
3,Miami Heat*,2005,82,243.7,37.8,77.7,0.486,5.8,15.4,0.377,...,13.1,27.0,0.260,0.460,12.4,74.1,0.242,AmericanAirlines Arena,"815,143","19,882"
4,Boston Celtics*,2005,82,242.4,37.1,79.4,0.468,5.3,15.3,0.349,...,14.7,27.2,0.273,0.481,14.2,70.1,0.274,FleetCenter,"656,081","16,002"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,Cleveland Cavaliers,2020,65,241.9,40.3,87.9,0.458,11.2,31.8,0.351,...,14.6,24.6,0.172,0.560,11.7,77.4,0.164,Quicken Loans Arena,"643,008","17,861"
476,Chicago Bulls,2020,65,241.2,39.6,88.6,0.447,12.2,35.1,0.348,...,13.7,22.8,0.175,0.546,16.3,75.6,0.239,United Center,"639,352","18,804"
477,Orlando Magic,2020,65,240.4,39.2,88.8,0.442,10.9,32.0,0.341,...,11.4,22.4,0.191,0.535,13.5,79.0,0.170,Amway Center,"529,870","17,093"
478,Golden State Warriors,2020,65,241.9,38.6,88.2,0.438,10.4,31.3,0.334,...,13.2,21.5,0.212,0.553,13.7,76.4,0.193,Chase Center,"614,176","18,064"


### Format All Data

In [6]:
# drop unwanted columns
outcomeData = outcomeData.drop(["Start Time", "Box Score", "Notes"], axis=1)

# change OT to integer value for number of OT rounds
outcomeData.loc[(outcomeData["OT?"].isna()),'OT?']= 0
outcomeData.loc[(outcomeData["OT?"] == "OT"),'OT?']= 1
outcomeData.loc[(outcomeData["OT?"] == "2OT"),'OT?']= 2
outcomeData.loc[(outcomeData["OT?"] == "3OT"),'OT?']= 3
outcomeData.loc[(outcomeData["OT?"] == "4OT"),'OT?']= 4


In [7]:
# function for redefining team names
def redefineTeamNames(dataframe):
    
    dataframe = dataframe.replace('Seattle SuperSonics', 'Oklahoma City Thunder', regex=True)
    
    dataframe = dataframe.replace('New Orleans/Oklahoma City Hornets', 'New Orleans Pelicans', regex=True)

    dataframe = dataframe.replace('New Orleans Hornets', 'New Orleans Pelicans', regex=True)
    
    dataframe = dataframe.replace('Charlotte Bobcats', 'Charlotte Hornets', regex=True)
    
    dataframe = dataframe.replace('New Jersey Nets', 'Brooklyn Nets', regex=True)

    return dataframe

In [8]:
# call function to redefine team names
outcomeData = redefineTeamNames(outcomeData)
teamData = redefineTeamNames(teamData)

# set up encoder and use one hot encoding to transform team names
encoding = LabelEncoder()
encoding.fit(outcomeData['Home'].values)
outcomeData['HomeEncoded'] = encoding.transform(outcomeData['Home'].values)
outcomeData['VisitorEncoded'] = encoding.transform(outcomeData['Visitor'].values)

In [9]:
# generate column names
teamDataColumn = teamData.drop(['Team', "Year", "Arena", "Attend.", "Attend./G", "G_PG", "MP_PG"], axis=1).columns
print(teamDataColumn)
testTeamDataColumn = teamData.drop(teamData.columns.difference(["3P_PG", "FGA_PG", "FG%_PG", "3P_PG", 
                                                               "3PA_PG", "3P%_PG", "ORB_PG", "DRB_PG", "TRB_PG", "AST_PG"
                                                                , "STL_PG", "BLK_PG", "TOV_PG", "2P_PG", "2PA_PG"]), axis=1).columns
print(testTeamDataColumn)
# outcomeData = outcomeData.loc[outcomeData['Year'] == 2020]
outcomeData



Index(['FG_PG', 'FGA_PG', 'FG%_PG', '3P_PG', '3PA_PG', '3P%_PG', '2P_PG',
       '2PA_PG', '2P%_PG', 'FT_PG',
       ...
       '3PAr', 'TS%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA', 'eFG%.1', 'TOV%.1',
       'DRB%', 'FT/FGA.1'],
      dtype='object', length=113)
Index(['FGA_PG', 'FG%_PG', '3P_PG', '3PA_PG', '3P%_PG', '2P_PG', '2PA_PG',
       'ORB_PG', 'DRB_PG', 'TRB_PG', 'AST_PG', 'STL_PG', 'BLK_PG', 'TOV_PG'],
      dtype='object')


,Year,Month,Visitor,VisitorPTS,Home,HomePTS,OT?,Attend,Total Points,Winner,HomeEncoded,VisitorEncoded
0,2005,november,Houston Rockets,79,Detroit Pistons,87,0,22076,166,Detroit Pistons,8,10
1,2005,november,Sacramento Kings,98,Dallas Mavericks,107,0,20041,205,Dallas Mavericks,6,25
2,2005,november,Denver Nuggets,78,Los Angeles Lakers,89,0,18997,167,Los Angeles Lakers,13,7
3,2005,november,Indiana Pacers,109,Cleveland Cavaliers,104,2,19730,213,Indiana Pacers,5,11
4,2005,november,Milwaukee Bucks,92,Orlando Magic,93,0,15138,185,Orlando Magic,21,16
...,...,...,...,...,...,...,...,...,...,...,...,...
20429,2020,march,Los Angeles Clippers,131,Golden State Warriors,107,0,18064,238,Los Angeles Clippers,9,12
20430,2020,march,Detroit Pistons,106,Philadelphia 76ers,124,0,20172,230,Philadelphia 76ers,22,8
20431,2020,march,New York Knicks,136,Atlanta Hawks,131,1,15393,267,New York Knicks,0,19
20432,2020,march,Charlotte Hornets,109,Miami Heat,98,0,19600,207,Charlotte Hornets,15,3


### Feature Engineering

#### y data

In [10]:
# function for finding if the home team is the winner
def homeWin(dataframe):
    if dataframe['Winner'] == dataframe['Home']:
        return True
    else:
        return False
    
# add in HomeWin column
outcomeData['HomeWin'] = outcomeData.apply(homeWin, axis=1)

In [11]:
won_last = defaultdict(int)

for index, row in outcomeData.iterrows():
    home_team = row["Home"]
    visitor_team = row["Visitor"]
    outcomeData.loc[index,'HomeLastWin'] = won_last[home_team]
    outcomeData.loc[index,'VisitorLastWin'] = won_last[visitor_team]
    won_last[home_team] = row["HomeWin"]
    won_last[visitor_team] = not row["HomeWin"]

In [12]:
outcomeData["HomeWinStreak"] = 0
outcomeData["VisitorWinStreak"] = 0

win_streak = defaultdict(int)

for index, row in outcomeData.iterrows():
    home_team = row["Home"]
    visitor_team = row["Visitor"]
    outcomeData.loc[index,'HomeWinStreak'] = win_streak[home_team]
    outcomeData.loc[index,'VisitorWinStreak'] = win_streak[visitor_team]
    if row["HomeWin"]:
        win_streak[home_team] += 1
        win_streak[visitor_team] = 0
    else:
        win_streak[home_team] = 0
        win_streak[visitor_team] += 1

In [13]:
outcomeData

,Year,Month,Visitor,VisitorPTS,Home,HomePTS,OT?,Attend,Total Points,Winner,HomeEncoded,VisitorEncoded,HomeWin,HomeLastWin,VisitorLastWin,HomeWinStreak,VisitorWinStreak
0,2005,november,Houston Rockets,79,Detroit Pistons,87,0,22076,166,Detroit Pistons,8,10,True,0,0,0,0
1,2005,november,Sacramento Kings,98,Dallas Mavericks,107,0,20041,205,Dallas Mavericks,6,25,True,0,0,0,0
2,2005,november,Denver Nuggets,78,Los Angeles Lakers,89,0,18997,167,Los Angeles Lakers,13,7,True,0,0,0,0
3,2005,november,Indiana Pacers,109,Cleveland Cavaliers,104,2,19730,213,Indiana Pacers,5,11,False,0,0,0,0
4,2005,november,Milwaukee Bucks,92,Orlando Magic,93,0,15138,185,Orlando Magic,21,16,True,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20429,2020,march,Los Angeles Clippers,131,Golden State Warriors,107,0,18064,238,Los Angeles Clippers,9,12,False,True,False,1,0
20430,2020,march,Detroit Pistons,106,Philadelphia 76ers,124,0,20172,230,Philadelphia 76ers,22,8,True,False,False,0,0
20431,2020,march,New York Knicks,136,Atlanta Hawks,131,1,15393,267,New York Knicks,0,19,False,True,False,1,0
20432,2020,march,Charlotte Hornets,109,Miami Heat,98,0,19600,207,Charlotte Hornets,15,3,False,True,False,1,0


In [15]:
def home_team_ranks_higher(row):
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    home_rank = ladder.loc[home_team]["Rk"]
    visitor_rank = ladder.loc[visitor_team]["Rk"]
    return home_rank < visitor_rank

In [16]:
# def home_team_won_last(row):
#     home_team = row["Home Team"]
#     visitor_team = row["Visitor Team"]
    
#     teams = tuple(sorted([home_team, visitor_team]))
#     result = 1 if last_match_winner[teams] == row["Home Team"] else 0
    
#     winner = row["Home Team"] if row["HomeWin"] else row["Visitor Team"]
#     last_match_winner[teams] = winner
    
#     return result
    
# outcomeData['HomeTeamWonLast'] = outcomeData.apply(home_team_won_last, axis=1)

In [17]:
# spread
outcomeData["Final Home Spread"] = (outcomeData["VisitorPTS"] - outcomeData["HomePTS"]).astype(float)

#### x data

In [20]:
def differentialVariableGenerator(outcomeData, teamData, teamDataColumn):
                                  
    global teamData_global
    global counter 
    global differentialFeatureData
    x = []
    for col in teamDataColumn:
        x.append(col + "_home")
        x.append(col + "_visitor")
    
    differentialFeatureData = pd.DataFrame(index=range(outcomeData.shape[0]), columns = x)
    print(outcomeData.shape[0])
    teamData_global = teamData
    totalCount = outcomeData.shape[0]
    counter = 0
    overallTime = time.time()
    
    def calcDifferences(row):
                                  
        global teamData_global
        global counter
                                  
        counter = counter + 1;
        start = time.time()
        year = row['Year']
        visitor = row['Visitor']
        home = row['Home']
                                  
        def columnCalc(col):
                                  
            variable = col.name
            var_home = variable+"_home"
            var_visitor = variable+"_visitor"
            year_subset = teamData_global.loc[(teamData_global.Year == year)]
            visitorValue = year_subset.loc[year_subset.Team.astype(str).str.contains(visitor)][variable]
            homeValue = year_subset.loc[year_subset.Team.astype(str).str.contains(home)][variable]
            #difference = homeValue.iloc[0] - visitorValue.iloc[0]
            differentialFeatureData[var_home][counter-1] = homeValue.iloc[0]
            #print(differentialFeatureData[var_home][counter-1])
            differentialFeatureData[var_visitor][counter-1] = visitorValue.iloc[0]
            #print(differentialFeatureData)
                                  
        teamData_global[teamDataColumn].apply(columnCalc)
        #print(differentialFeatureData)
#         print(visitor, home, time.time() - start,time.time() - overallTime)
        print("{count}/{total}".format(count = counter, total = totalCount))
                                  
    outcomeData.apply(calcDifferences, axis=1)
#     print("Total time {time} seconds".format(time = time.time() - overallTime))
    return differentialFeatureData

In [21]:
featureData = differentialVariableGenerator(outcomeData, teamData, testTeamDataColumn)

20434
1/20434
2/20434
3/20434
4/20434
5/20434
6/20434
7/20434
8/20434
9/20434
10/20434
11/20434
12/20434
13/20434
14/20434
15/20434
16/20434
17/20434
18/20434
19/20434
20/20434
21/20434
22/20434
23/20434
24/20434
25/20434
26/20434
27/20434
28/20434
29/20434
30/20434
31/20434
32/20434
33/20434
34/20434
35/20434
36/20434
37/20434
38/20434
39/20434
40/20434
41/20434
42/20434
43/20434
44/20434
45/20434
46/20434
47/20434
48/20434
49/20434
50/20434
51/20434
52/20434
53/20434
54/20434
55/20434
56/20434
57/20434
58/20434
59/20434
60/20434
61/20434
62/20434
63/20434
64/20434
65/20434
66/20434
67/20434
68/20434
69/20434
70/20434
71/20434
72/20434
73/20434
74/20434
75/20434
76/20434
77/20434
78/20434
79/20434
80/20434
81/20434
82/20434
83/20434
84/20434
85/20434
86/20434
87/20434
88/20434
89/20434
90/20434
91/20434
92/20434
93/20434
94/20434
95/20434
96/20434
97/20434
98/20434
99/20434
100/20434
101/20434
102/20434
103/20434
104/20434
105/20434
106/20434
107/20434
108/20434
109/20434
110/20434
11

830/20434
831/20434
832/20434
833/20434
834/20434
835/20434
836/20434
837/20434
838/20434
839/20434
840/20434
841/20434
842/20434
843/20434
844/20434
845/20434
846/20434
847/20434
848/20434
849/20434
850/20434
851/20434
852/20434
853/20434
854/20434
855/20434
856/20434
857/20434
858/20434
859/20434
860/20434
861/20434
862/20434
863/20434
864/20434
865/20434
866/20434
867/20434
868/20434
869/20434
870/20434
871/20434
872/20434
873/20434
874/20434
875/20434
876/20434
877/20434
878/20434
879/20434
880/20434
881/20434
882/20434
883/20434
884/20434
885/20434
886/20434
887/20434
888/20434
889/20434
890/20434
891/20434
892/20434
893/20434
894/20434
895/20434
896/20434
897/20434
898/20434
899/20434
900/20434
901/20434
902/20434
903/20434
904/20434
905/20434
906/20434
907/20434
908/20434
909/20434
910/20434
911/20434
912/20434
913/20434
914/20434
915/20434
916/20434
917/20434
918/20434
919/20434
920/20434
921/20434
922/20434
923/20434
924/20434
925/20434
926/20434
927/20434
928/20434
929/20434


1593/20434
1594/20434
1595/20434
1596/20434
1597/20434
1598/20434
1599/20434
1600/20434
1601/20434
1602/20434
1603/20434
1604/20434
1605/20434
1606/20434
1607/20434
1608/20434
1609/20434
1610/20434
1611/20434
1612/20434
1613/20434
1614/20434
1615/20434
1616/20434
1617/20434
1618/20434
1619/20434
1620/20434
1621/20434
1622/20434
1623/20434
1624/20434
1625/20434
1626/20434
1627/20434
1628/20434
1629/20434
1630/20434
1631/20434
1632/20434
1633/20434
1634/20434
1635/20434
1636/20434
1637/20434
1638/20434
1639/20434
1640/20434
1641/20434
1642/20434
1643/20434
1644/20434
1645/20434
1646/20434
1647/20434
1648/20434
1649/20434
1650/20434
1651/20434
1652/20434
1653/20434
1654/20434
1655/20434
1656/20434
1657/20434
1658/20434
1659/20434
1660/20434
1661/20434
1662/20434
1663/20434
1664/20434
1665/20434
1666/20434
1667/20434
1668/20434
1669/20434
1670/20434
1671/20434
1672/20434
1673/20434
1674/20434
1675/20434
1676/20434
1677/20434
1678/20434
1679/20434
1680/20434
1681/20434
1682/20434
1683/20434

2340/20434
2341/20434
2342/20434
2343/20434
2344/20434
2345/20434
2346/20434
2347/20434
2348/20434
2349/20434
2350/20434
2351/20434
2352/20434
2353/20434
2354/20434
2355/20434
2356/20434
2357/20434
2358/20434
2359/20434
2360/20434
2361/20434
2362/20434
2363/20434
2364/20434
2365/20434
2366/20434
2367/20434
2368/20434
2369/20434
2370/20434
2371/20434
2372/20434
2373/20434
2374/20434
2375/20434
2376/20434
2377/20434
2378/20434
2379/20434
2380/20434
2381/20434
2382/20434
2383/20434
2384/20434
2385/20434
2386/20434
2387/20434
2388/20434
2389/20434
2390/20434
2391/20434
2392/20434
2393/20434
2394/20434
2395/20434
2396/20434
2397/20434
2398/20434
2399/20434
2400/20434
2401/20434
2402/20434
2403/20434
2404/20434
2405/20434
2406/20434
2407/20434
2408/20434
2409/20434
2410/20434
2411/20434
2412/20434
2413/20434
2414/20434
2415/20434
2416/20434
2417/20434
2418/20434
2419/20434
2420/20434
2421/20434
2422/20434
2423/20434
2424/20434
2425/20434
2426/20434
2427/20434
2428/20434
2429/20434
2430/20434

3090/20434
3091/20434
3092/20434
3093/20434
3094/20434
3095/20434
3096/20434
3097/20434
3098/20434
3099/20434
3100/20434
3101/20434
3102/20434
3103/20434
3104/20434
3105/20434
3106/20434
3107/20434
3108/20434
3109/20434
3110/20434
3111/20434
3112/20434
3113/20434
3114/20434
3115/20434
3116/20434
3117/20434
3118/20434
3119/20434
3120/20434
3121/20434
3122/20434
3123/20434
3124/20434
3125/20434
3126/20434
3127/20434
3128/20434
3129/20434
3130/20434
3131/20434
3132/20434
3133/20434
3134/20434
3135/20434
3136/20434
3137/20434
3138/20434
3139/20434
3140/20434
3141/20434
3142/20434
3143/20434
3144/20434
3145/20434
3146/20434
3147/20434
3148/20434
3149/20434
3150/20434
3151/20434
3152/20434
3153/20434
3154/20434
3155/20434
3156/20434
3157/20434
3158/20434
3159/20434
3160/20434
3161/20434
3162/20434
3163/20434
3164/20434
3165/20434
3166/20434
3167/20434
3168/20434
3169/20434
3170/20434
3171/20434
3172/20434
3173/20434
3174/20434
3175/20434
3176/20434
3177/20434
3178/20434
3179/20434
3180/20434

...

17391/20434
17392/20434
17393/20434
17394/20434
17395/20434
17396/20434
17397/20434
17398/20434
17399/20434
17400/20434
17401/20434
17402/20434
17403/20434
17404/20434
17405/20434
17406/20434
17407/20434
17408/20434
17409/20434
17410/20434
17411/20434
17412/20434
17413/20434
17414/20434
17415/20434
17416/20434
17417/20434
17418/20434
17419/20434
17420/20434
17421/20434
17422/20434
17423/20434
17424/20434
17425/20434
17426/20434
17427/20434
17428/20434
17429/20434
17430/20434
17431/20434
17432/20434
17433/20434
17434/20434
17435/20434
17436/20434
17437/20434
17438/20434
17439/20434
17440/20434
17441/20434
17442/20434
17443/20434
17444/20434
17445/20434
17446/20434
17447/20434
17448/20434
17449/20434
17450/20434
17451/20434
17452/20434
17453/20434
17454/20434
17455/20434
17456/20434
17457/20434
17458/20434
17459/20434
17460/20434
17461/20434
17462/20434
17463/20434
17464/20434
17465/20434
17466/20434
17467/20434
17468/20434
17469/20434
17470/20434
17471/20434
17472/20434
17473/20434
1747

18076/20434
18077/20434
18078/20434
18079/20434
18080/20434
18081/20434
18082/20434
18083/20434
18084/20434
18085/20434
18086/20434
18087/20434
18088/20434
18089/20434
18090/20434
18091/20434
18092/20434
18093/20434
18094/20434
18095/20434
18096/20434
18097/20434
18098/20434
18099/20434
18100/20434
18101/20434
18102/20434
18103/20434
18104/20434
18105/20434
18106/20434
18107/20434
18108/20434
18109/20434
18110/20434
18111/20434
18112/20434
18113/20434
18114/20434
18115/20434
18116/20434
18117/20434
18118/20434
18119/20434
18120/20434
18121/20434
18122/20434
18123/20434
18124/20434
18125/20434
18126/20434
18127/20434
18128/20434
18129/20434
18130/20434
18131/20434
18132/20434
18133/20434
18134/20434
18135/20434
18136/20434
18137/20434
18138/20434
18139/20434
18140/20434
18141/20434
18142/20434
18143/20434
18144/20434
18145/20434
18146/20434
18147/20434
18148/20434
18149/20434
18150/20434
18151/20434
18152/20434
18153/20434
18154/20434
18155/20434
18156/20434
18157/20434
18158/20434
1815

18766/20434
18767/20434
18768/20434
18769/20434
18770/20434
18771/20434
18772/20434
18773/20434
18774/20434
18775/20434
18776/20434
18777/20434
18778/20434
18779/20434
18780/20434
18781/20434
18782/20434
18783/20434
18784/20434
18785/20434
18786/20434
18787/20434
18788/20434
18789/20434
18790/20434
18791/20434
18792/20434
18793/20434
18794/20434
18795/20434
18796/20434
18797/20434
18798/20434
18799/20434
18800/20434
18801/20434
18802/20434
18803/20434
18804/20434
18805/20434
18806/20434
18807/20434
18808/20434
18809/20434
18810/20434
18811/20434
18812/20434
18813/20434
18814/20434
18815/20434
18816/20434
18817/20434
18818/20434
18819/20434
18820/20434
18821/20434
18822/20434
18823/20434
18824/20434
18825/20434
18826/20434
18827/20434
18828/20434
18829/20434
18830/20434
18831/20434
18832/20434
18833/20434
18834/20434
18835/20434
18836/20434
18837/20434
18838/20434
18839/20434
18840/20434
18841/20434
18842/20434
18843/20434
18844/20434
18845/20434
18846/20434
18847/20434
18848/20434
1884

19453/20434
19454/20434
19455/20434
19456/20434
19457/20434
19458/20434
19459/20434
19460/20434
19461/20434
19462/20434
19463/20434
19464/20434
19465/20434
19466/20434
19467/20434
19468/20434
19469/20434
19470/20434
19471/20434
19472/20434
19473/20434
19474/20434
19475/20434
19476/20434
19477/20434
19478/20434
19479/20434
19480/20434
19481/20434
19482/20434
19483/20434
19484/20434
19485/20434
19486/20434
19487/20434
19488/20434
19489/20434
19490/20434
19491/20434
19492/20434
19493/20434
19494/20434
19495/20434
19496/20434
19497/20434
19498/20434
19499/20434
19500/20434
19501/20434
19502/20434
19503/20434
19504/20434
19505/20434
19506/20434
19507/20434
19508/20434
19509/20434
19510/20434
19511/20434
19512/20434
19513/20434
19514/20434
19515/20434
19516/20434
19517/20434
19518/20434
19519/20434
19520/20434
19521/20434
19522/20434
19523/20434
19524/20434
19525/20434
19526/20434
19527/20434
19528/20434
19529/20434
19530/20434
19531/20434
19532/20434
19533/20434
19534/20434
19535/20434
1953

20137/20434
20138/20434
20139/20434
20140/20434
20141/20434
20142/20434
20143/20434
20144/20434
20145/20434
20146/20434
20147/20434
20148/20434
20149/20434
20150/20434
20151/20434
20152/20434
20153/20434
20154/20434
20155/20434
20156/20434
20157/20434
20158/20434
20159/20434
20160/20434
20161/20434
20162/20434
20163/20434
20164/20434
20165/20434
20166/20434
20167/20434
20168/20434
20169/20434
20170/20434
20171/20434
20172/20434
20173/20434
20174/20434
20175/20434
20176/20434
20177/20434
20178/20434
20179/20434
20180/20434
20181/20434
20182/20434
20183/20434
20184/20434
20185/20434
20186/20434
20187/20434
20188/20434
20189/20434
20190/20434
20191/20434
20192/20434
20193/20434
20194/20434
20195/20434
20196/20434
20197/20434
20198/20434
20199/20434
20200/20434
20201/20434
20202/20434
20203/20434
20204/20434
20205/20434
20206/20434
20207/20434
20208/20434
20209/20434
20210/20434
20211/20434
20212/20434
20213/20434
20214/20434
20215/20434
20216/20434
20217/20434
20218/20434
20219/20434
2022

In [22]:
featureData.to_csv('/Users/justinholmes/Desktop/Sports_Model/Data/feature_data.csv')
featureData

,FGA_PG_home,FGA_PG_visitor,FG%_PG_home,FG%_PG_visitor,3P_PG_home,3P_PG_visitor,3PA_PG_home,3PA_PG_visitor,3P%_PG_home,3P%_PG_visitor,...,TRB_PG_home,TRB_PG_visitor,AST_PG_home,AST_PG_visitor,STL_PG_home,STL_PG_visitor,BLK_PG_home,BLK_PG_visitor,TOV_PG_home,TOV_PG_visitor
0,78.3,78.3,0.444,0.443,4.4,6.7,12.8,18.5,0.345,0.364,...,43.4,42.4,21.8,21.1,7,6.9,6.1,4.6,13.8,13.8
1,81.6,85.1,0.457,0.459,5.6,6.4,15.5,17,0.364,0.374,...,42.9,42.4,19.6,24.5,8.6,8.2,5.6,3.9,13.4,13.1
2,80.8,80.7,0.437,0.459,7.9,3.9,22.1,11.5,0.355,0.34,...,43.2,42,20.4,23.9,6.2,9.1,4.2,6,14.3,14.9
3,81.5,75.2,0.447,0.432,3.7,6.6,11,19.2,0.332,0.344,...,42.3,40.1,22.6,18.2,8,7.5,5.6,4.4,13.9,14.3
4,81.5,80.2,0.454,0.45,3.9,4,11.2,11.3,0.349,0.351,...,43.7,41.3,19.3,21,7.7,6.5,5.4,3.5,16.1,13.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20429,88.2,89.7,0.438,0.464,10.4,12.2,31.3,33.2,0.334,0.366,...,42.8,48,25.6,23.8,8.2,7.1,4.6,5,14.9,14.8
20430,87.7,85.7,0.465,0.459,11.4,12,31.6,32.7,0.362,0.367,...,45.5,41.7,25.9,24.1,8.2,7.4,5.4,4.5,14.2,15.3
20431,90.6,89.3,0.449,0.447,12,9.6,36.1,28.4,0.333,0.337,...,43.3,46.5,24,22.1,7.8,7.6,5.1,4.7,16.2,14.3
20432,84.4,85.9,0.47,0.434,13.4,12.1,34.8,34.3,0.383,0.352,...,44.5,42.8,26,23.8,7.4,6.6,4.5,4.1,14.9,14.6


In [ ]:
x_columns = []
for col in teamDataColumn:
    x_columns.append(col + "_DIFF")
print(x_columns)

### Training Model

In [ ]:
# target column (in this case, whether or not the home team will win)
y = outcomeData["HomeWin"]
# y = outcomeData["Final Home Spread"]

# columns that will be used to make the prediction
# X = outcomeData[x_columns]
# X = testOutcomeData
X_previous_wins = outcomeData[["HomeLastWin", "VisitorLastWin"]].values
X_win_streak = outcomeData[["HomeLastWin", "VisitorLastWin", "HomeWinStreak", "VisitorWinStreak"]].values

# X = outcomeData[["HomeLastWin", "VisitorLastWin", "HomeTeamRanksHigher", "HomeTeamWonLast"]].values

# set up scorer for testing accuracy
scorer = make_scorer(f1_score, pos_label = None, average = "weighted")

In [ ]:
# Set up train test split
# X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y, test_size=0.20, random_state=100)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=100)

In [ ]:
y_true = outcomeData["HomeWin"].values

# cross val score
clf = RandomForestClassifier(random_state = 14)
#scores = cross_val_score(clf, X_train, y_train, scoring=scorer)
scores_previous_wins = cross_val_score(clf, X_previous_wins, y_true, scoring=scorer)
scores_win_streak = cross_val_score(clf, X_win_streak, y_true, scoring=scorer)

In [ ]:
# mean score
y_pred = [1] * len(y_true)

print("F1 Score to Beat: {:.4f}".format(f1_score(y_true, y_pred, pos_label=None, average='weighted')))

print("F1 Score with Previous Wins: {:.4f}".format(np.mean(scores_previous_wins)))

print("F1 Score with Win Streaks: {:.4f}".format(np.mean(scores_win_streak)))
